In [17]:
import numpy as np

files = {0:'me_at_the_zoo.in', 1:'videos_worth_spreading.in', 2:'trending_today.in', 3:'kittens.in'} 

In [61]:
def load_data(id_file_to_load = 0, print_debug = True):
    # Load file
    file = open(files[id_file_to_load], 'r') 

    # Header / Video / Endpoints / request descr. / Cache
    header = file.readline().replace('\n','')
    header = np.fromstring(header, dtype=int, sep=' ')
    header_dict = {   'n_video':header[0], 'n_endpoint':header[1], 
                      'requ_des':header[2], 'n_cache':header[3],
                      'size_cache':header[4]}

    # Video size in MB
    video_size = file.readline().replace('\n','')
    video_size = np.fromstring(video_size, dtype=int, sep=' ')

    # Endpoints locations and connections
    endpoint_cache = np.zeros((header_dict['n_endpoint'], header_dict['n_cache'])) # Latency endpoint to cache
    endpoint_datacenter = np.zeros(header_dict['n_endpoint']) # Latency endpoint to datacenter
    endpoint_video = np.zeros((header_dict['n_endpoint'], header_dict['n_video'])) # Endpoints requests

    # Reding endpoint to cache latency and endpoint to data_center latency
    for id_endpoint in range(header_dict['n_endpoint']):
        endpoint = file.readline().replace('\n','').split(' ')
        endpoint_datacenter[id_endpoint] = int(endpoint[0])
        n_latency = int(endpoint[1])
        for i in range(n_latency):
            lat_cache = file.readline().replace('\n','').split(' ')
            endpoint_cache[id_endpoint, int(lat_cache[0])] = int(lat_cache[1])

    for id_requ_des in range(header_dict['requ_des']):
        requ_des = file.readline().replace('\n','').split(' ')
        endpoint_video[int(requ_des[1]), int(requ_des[0])] += int(requ_des[2])

    if print_debug:
        print('File description=\n', '\t\t', header_dict)
        print('Video sizes=\n', '\t\tshape=', video_size.shape, ' nnz=', len(np.nonzero(video_size)[0]))
        print('Endpoint to chache=\n', '\t\tshape=', endpoint_cache.shape, ' nnz=', len(np.nonzero(endpoint_cache)[0]))
        print('Endpoint to video=\n', '\t\tshape=', endpoint_video.shape, ' max=', np.max(endpoint_video))
    
    return endpoint_cache, endpoint_datacenter, endpoint_video, video_size, header_dict["size_cache"]

endpoint_cache, endpoint_datacenter, endpoint_video, video_size, size_cache  = load_data(id_file_to_load=0)

File description=
 		 {'n_endpoint': 10, 'size_cache': 100, 'n_cache': 10, 'n_video': 100, 'requ_des': 100}
Video sizes=
 		shape= (100,)  nnz= 100
Endpoint to chache=
 		shape= (10, 10)  nnz= 32
Endpoint to video=
 		shape= (10, 100)  max= 3296.0


In [15]:
endpoint_cache[:][0]

array([ 170.,   22.,  224.,    0.,    0.,    0.,    0.,    0.,    0.,    0.])

In [19]:
cache_videos_saved = np.zeros((endpoint_cache.shape[1], video_size.shape[0])) # Latency endpoint to cache
cache_videos_saved.shape

(10, 100)

In [42]:
endpoint_video[5][0]

0.0

In [46]:
endpoint_cache[9][0]

0.0

In [47]:
for i in range(endpoint_cache.shape[1]):
    for j in range(endpoint_cache.shape[0]):
        if endpoint_cache[j][i] > 0:
            for k in range(video_size.shape[0]):
                if(endpoint_video[j][k] > 0):
                    print("saved time on video", k, "with cache", i, "with endpoint", j)
                cache_videos_saved[i][j] += (endpoint_datacenter[j] - endpoint_cache[j][i]) * endpoint_video[j][k]

saved time on video 1 with cache 0 with endpoint 0
saved time on video 7 with cache 0 with endpoint 0
saved time on video 8 with cache 0 with endpoint 0
saved time on video 13 with cache 0 with endpoint 0
saved time on video 15 with cache 0 with endpoint 0
saved time on video 26 with cache 0 with endpoint 0
saved time on video 31 with cache 0 with endpoint 0
saved time on video 65 with cache 0 with endpoint 0
saved time on video 99 with cache 0 with endpoint 0
saved time on video 0 with cache 0 with endpoint 1
saved time on video 1 with cache 0 with endpoint 1
saved time on video 5 with cache 0 with endpoint 1
saved time on video 7 with cache 0 with endpoint 1
saved time on video 10 with cache 0 with endpoint 1
saved time on video 46 with cache 0 with endpoint 1
saved time on video 89 with cache 0 with endpoint 1
saved time on video 1 with cache 0 with endpoint 5
saved time on video 2 with cache 0 with endpoint 5
saved time on video 5 with cache 0 with endpoint 5
saved time on video 8 

In [48]:
cache_videos_saved

array([[  8441802.,   8007558.,         0.,         0.,         0.,
          3169480.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,         0.,         0.,
                0.,         0.,         0.,     

In [50]:
import knapsack as ks

In [65]:
items_value = cache_videos_saved[0].astype(int)
items_weight = video_size
items = list(zip(items_weight,items_value))
print(items)

[(20, 8441802), (11, 8007558), (50, 0), (26, 0), (5, 0), (3, 3169480), (6, 0), (32, 0), (40, 0), (22, 0), (4, 0), (20, 0), (50, 0), (27, 0), (49, 0), (44, 0), (1, 0), (37, 0), (35, 0), (27, 0), (14, 0), (33, 0), (6, 0), (22, 0), (23, 0), (48, 0), (44, 0), (14, 0), (26, 0), (9, 0), (46, 0), (44, 0), (15, 0), (32, 0), (31, 0), (8, 0), (39, 0), (27, 0), (39, 0), (27, 0), (1, 0), (17, 0), (1, 0), (47, 0), (44, 0), (42, 0), (16, 0), (3, 0), (44, 0), (48, 0), (5, 0), (25, 0), (4, 0), (39, 0), (39, 0), (7, 0), (24, 0), (28, 0), (14, 0), (44, 0), (22, 0), (11, 0), (27, 0), (37, 0), (11, 0), (16, 0), (50, 0), (33, 0), (22, 0), (26, 0), (7, 0), (12, 0), (17, 0), (30, 0), (12, 0), (12, 0), (4, 0), (32, 0), (12, 0), (46, 0), (43, 0), (4, 0), (12, 0), (34, 0), (11, 0), (7, 0), (47, 0), (29, 0), (24, 0), (40, 0), (41, 0), (10, 0), (5, 0), (22, 0), (22, 0), (24, 0), (37, 0), (34, 0), (50, 0), (5, 0)]


In [66]:
solution, w, v = ks.solve(items, size_cache)

5
0
19618840
19618840
50
0
19618840
19618840
34
0
19618840
19618840
37
0
19618840
19618840
24
0
19618840
19618840
22
0
19618840
19618840
22
0
19618840
19618840
5
0
19618840
19618840
10
0
19618840
19618840
41
0
19618840
19618840
40
0
19618840
19618840
24
0
19618840
19618840
29
0
19618840
19618840
47
0
19618840
19618840
7
0
19618840
19618840
11
0
19618840
19618840
34
0
19618840
19618840
12
0
19618840
19618840
4
0
19618840
19618840
43
0
19618840
19618840
46
0
19618840
19618840
12
0
19618840
19618840
32
0
19618840
19618840
4
0
19618840
19618840
12
0
19618840
19618840
12
0
19618840
19618840
30
0
19618840
19618840
17
0
19618840
19618840
12
0
19618840
19618840
7
0
19618840
19618840
26
0
19618840
19618840
22
0
19618840
19618840
33
0
19618840
19618840
50
0
19618840
19618840
16
0
19618840
19618840
11
0
19618840
19618840
37
0
19618840
19618840
27
0
19618840
19618840
11
0
19618840
19618840
22
0
19618840
19618840
44
0
19618840
19618840
14
0
19618840
19618840
28
0
19618840
19618840
24
0
19618840
196

In [67]:
if solution != []:
    
    print("solution of weight %d, value %d found:" % (w,v))
    print(", ".join(map(lambda t: str(t), solution)))
else:
    # this should never happen (and hasen't as far as we're concerned)
    print('no solution found(!).')

solution of weight 34, value 19618840 found:
(3, 3169480), (11, 8007558), (20, 8441802)
